# Cosas por agregar
- visualización
- filtrado
- incluir resumen de formas de indexado

# Links de interés
- Datasets https://www.kaggle.com/datasets
- Documentación de Pandas https://pandas.pydata.org/docs/index.html 
- Cookbook de pandas https://pandas.pydata.org/docs/user_guide/cookbook.html
- Estructura y un par de cosas sobre data engineering https://apmonitor.com/pds/index.php/Main/DataPreparation
- Indexing and selecting data, pandas: https://pandas.pydata.org/docs/user_guide/indexing.html#

# Import importantes y cargo pickles de dfs

In [ ]:
import kagglehub
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import datetime
import pickle


# Download latest version
kagglehub.dataset_download("isathyam31/adult-income-prediction-classification")


In [2]:
#Cargo pickles
with open("df.pkl","rb") as file:
    df=pickle.load(file)
    
with open("df_t.pkl","rb") as file:
    df_t=pickle.load(file)

# Recolectar data

## Datos en general

La función más usada, lejos es `read_csv` de pandas, donde basta que demos el path del archivo y su nombre, y se leerá si no hay mayor problema. Por default, la primera fila se transforma en las columnas del dataframe, que podemos ver usando `dataframe.columns`

In [ ]:
path="/home/capitanespiral/Documents/GitHub/data_science" #Aquí introducir tu path
filename="data.csv"

df=pd.read_csv(f"{path}/{filename}")
#Display entrega mucho mejor resultado que print para los dataframes!
display(df)
print(df.columns)

---
Opciones más usadas de `read_csv`:
- **sep o delimiter** &rarr; El separador de la data, usualmente se encuentra solo pero a veces es mejor darlo explícitamente. (en el caso que se confunda el identificador automático)
- **header** &rarr; Número de fila que se toma como nombre de columnas y desde el cual comienza la data, por default, cero (inicio del .csv).
- **skiprows** &rarr; Filas a saltar desde el inicio del archivo (indexeando desde cero, se puede entregar lista o tupla)
- **skipfooter** &rarr; Filas a saltar desde el final del archivo.
- **usecols** &rarr; Subset de columnas a usar (secuencia de números o nombres explícitos).

In [ ]:
#Si la primera fila está mala, cambiamos el header
df1=pd.read_csv("data_bad_first_row.csv") #Salió pésimo
df2=pd.read_csv("data_bad_first_row.csv",header=1) #Mucho mejor

display(df1)
display(df2)

In [ ]:
#Si la segunda está mala
df3=pd.read_csv("data_bad_second_row.csv") #Primer dato pésimo
df4=pd.read_csv("data_bad_second_row.csv",skiprows=[1],header=0) #Nos saltamos la segunda fila y conservamos el header

display(df3)
display(df4)

In [ ]:
#Seleccionemos ciertas columnas no más
df5=pd.read_csv("data_bad_first_row.csv",header=1,usecols=[0,3]) #Puede ser con números, aquí la primera y la cuarta
df6=pd.read_csv("data_bad_first_row.csv",header=1,usecols=range(4)) #Puede ser con iteradores, acá me entrega de la primera A la cuarta
df7=pd.read_csv("data_bad_first_row.csv",header=1,usecols=["age","education","workclass"]) #Puede ser con los nombres explícitos

display(df5)
display(df6)
display(df7)

In [ ]:
del df1,df2,df3,df4,df5,df6,df7

## Series de Tiempo

Para el caso de series de tiempo, hay que saber trabajar con `Timestamps`, `Datetime`, `Timedelta`, etc...

### Funciones generales 

- **pd.to_datetime()** &rarr; Transforma lo que le entregues en un "Datetime", funciona aceptando varios formatos y si le entregas secuencias.
- **pd.date_range()** &rarr; Para crear puntos en el tiempo equiespaceados, con un "start" un "end", posibilidad de "periods" o "freq"
    - **freq** más comunes:"y" (años),"m" (meses),"W" (semana), "B" (business day), "d" (días), "h" (horas), "min" (minutos), "s" (segundos) Ojo que hay *infinitas* opciones! revisar en https://pandas.pydata.org/docs/user_guide/timeseries.html#dateoffset-objects
- **df.resample().func()** &rarr; Entregando como variable una frecuencia, podemos "resamplear" según la función "func"

Ejemplos de estas funciones:

In [ ]:
##datetime con varios formatos
dt1=pd.to_datetime(["13/1/2018", np.datetime64("2018-01-13"), datetime.datetime(2018, 1, 13)],dayfirst=True)
print("dt1:",dt1,"\n")

In [ ]:
##date_range
#Creando una lista de tiempo cada tres horas
t1="1-1-2000"
t2="2000-3-10"
dt2=pd.date_range(start=t1,end=t2,freq="3h")
print("dt2:",dt2,"\n")

#Lo mismo pero con 15 minutos
t1="1-1-2000"
t2="2000-3-10"
dt3=pd.date_range(start=t1,end=t2,freq="15min")
print("dt3:",dt3,"\n")

#Si tengo periodos y frecuencia
dt4=pd.date_range(start="2018-8-1", periods=5, freq="2d")
print("dt4:",dt4,"\n")

In [ ]:
#Como usar el resampleo
idx = pd.date_range("2018-01-01", periods=10, freq="h") #ojo que 1h = h
ts = pd.Series(range(len(idx)), index=idx)
print("ts",ts,"\n",sep="\n")
#Downsample (también muy usado el sum)
ts_downsampled=ts.resample("2h").mean() #LEJOS el más útil
print("ts_downsampled",ts_downsampled,"\n",sep="\n")
#upsample (también muy usado bfill)
ts_upsampled=ts.resample("30min").ffill() #Sería interesante, acá interpolar
print("ts_upsampled",ts_upsampled,"\n",sep="\n")

In [16]:
del dt1,dt2,dt3,dt4,idx,ts,ts_upsampled,ts_downsampled

Ahora trabajemos con data de verdad:

In [ ]:
df_t=pd.read_csv("biomet1.csv",skiprows=[1],header=0) #Tenemos caso de segunda linea sin sentido
df_t["time_t"]=pd.to_datetime(df_t['date']+" "+df_t['time']) #Lo guardamos en una nueva columna
display(df_t)
print("\nLa columna nueva:",df_t["time_t"],sep="\n")

Nos servirá ahora
- **datetime.min() y .max()** &rarr; Evidente
- **datetimeindex.atributos** &rarr; nos permite acceder a muchas funciones de tiempo como "hour","day","minute","second","dayofyear", "strftime", entre otras (si es una serie, no un índice, agregar dt). La totalidad de atributos y métodos en https://pandas.pydata.org/docs/reference/api/pandas.DatetimeIndex.html. La totalidad de formatos para "strftime" en https://docs.python.org/3/library/datetime.html#strftime-and-strptime-behavior

In [ ]:
#Podemos acceder a propiedades o métodos específicos de un objeto tipo "datetime" usando el "dt"
print("Solo el año:",df_t["time_t"].dt.year,"\n",sep="\n")
print("Cambio el formato:",df_t["time_t"].dt.strftime("%Y  /   %m   /  %d"),"\n",sep="\n")

In [ ]:
#Fijamos un nuevo índice
df_t=df_t.set_index("time_t") #Esto "traslada" la columna "time_t" (ya no existe como columna) - Y pasa a ser datetimeindex
df_t=df_t.drop(columns = ["date","time"]) #Boto las columnas que ya no necesito
display(df_t)
print("Y el nuevo índice:")
print(df_t.index)

In [ ]:
#Al fijarlo como índice, ahora podemos accesar simplemente con ".atributo"
display(df_t)
print("Solo el día:",df_t.index.day,"\n",sep="\n")
print("Solo el día juliano:",df_t.index.dayofyear,"\n",sep="\n")

In [ ]:
#Podemos notar que hay vaciós en la data, en específico 3!
fig,ax=plt.subplots(figsize=(16,10))
ax.scatter(df_t.index,df_t.index,s=5)
plt.show()

In [ ]:
#Ahora cubrimos todo el espacio
ti=df_t.index.min()
tf=df_t.index.max()
time_total=pd.date_range(start=ti,end=tf,freq="30min")
df_t=df_t.reindex(time_total) #Esta función redefine el índice, conservando datos y agregando nans si el nuevo índice coincide con el anterior o si no existe (para cada fila!)
display(df_t)

In [ ]:
#Se rellenó
fig,ax=plt.subplots(figsize=(16,10))
ax.scatter(df_t.index,df_t.index,s=5)
plt.show()

## Almacenamos como pickle los dataframes

In [20]:
#El primer dataframe
with open('df.pkl',"wb") as file:
    pickle.dump(df,file)

#El dataframe de serie de tiempo
with open('df_t.pkl',"wb") as file:
    pickle.dump(df_t,file)

# Ver data y estadística básica

## Indexación básica

Primero entender que *pandas* tiene dos grandes tipos de objetos: `Series` y `DataFrame`. El primero es como un "vector" (unidimensional) y el segundo está compuesto por varias `Series`, siendo como una "matriz". Y cada uno siempre tiene asociado su `Index`, que puede ser numérico o de cualquier naturaleza (uno común, temporal).

La forma básica de indexar cada tipo tiene distintos resultados:
- **Series[label]** &rarr; Dará un valor escalar en tal posición acorde al `Index`.
- **DataFrame[colname]** &rarr; Dará una *Serie* correspondiente a tal nombre de columna

In [ ]:
#Accedo a una columna, me entrega a una serie, y despues a un elemento en específico
print("**Dataframe**")
print("Edad",df["age"],"\n",sep="\n")
print("Tercer valor:",df["age"][2],"\n") #df["age"] es una serie, entonces df["age"][2] es el valor asociado al indice 2 de la serie

#Si el índice no es numérico, esto igual se puede hacer, pero se recomienda indexar acorde al índice
print("**Dataframe temporal**")
print("Calor sensible",df_t["H"],"\n",sep="\n") #Notar que el índice es temporal
print("Tercer valor indexando con 'int':",df_t["H"][2]) #Obtengo el tercer valor -> Notar advertencia de pandas!
print("Tercer valor indexando con un 'label' del 'index':",df_t["H"]["2014-01-01 01:30:00"]) #Lo mismo pero con el tiempo, engorroso pero preciso

Siempre podemos acceder a **varios valores** de una *serie* (con indexación típica de python) o **varias columnas** en un *dataframe* (agregando un nuevo []:). Notar que esto último entrega un nuevo `DataFrame`.

In [ ]:
display(df[["age","education"]])
display(df[["age","education"]][1:5]) 
print("Del segundo al quinto valor de una columna:",df["age"][1:5],sep="\n")

También usando slices de enteros en una *serie* o *dataframe* (en este caso se accesarán filas, no columnas)

In [ ]:
#Indexado clásico de python sobre una serie
print("Del primer al quinto dato:",df["age"][:5],"\n",sep="\n") 
print("Los datos pares:",df["age"][::2],"\n",sep="\n")

#Indexado clásico de python sobre un dataframe
print("\nDel primer al quinto dato:")
display(df_t[:5]) 
print("\nLos datos dados vuelta:")
display(df_t[::-1])

Para seleccionar algunas posiciones basta con entregar una lista, pero solo funciona con series!

In [ ]:
print("Segundo, tercer y sexto dato de 'age':")
print(df["age"][[1,2,5]])

print("\nSegundo, tercer y sexto dato de 'H':")
df_t["H"][[1,2,5]] #Notar el warning

## Indexar con .loc e .iloc

Para indexar también se tienen los métodos `.loc` y `.iloc`(para *series* y *dataframes*)
1. `.loc` &rarr; trabaja con labels o con arreglos booleanos.
2. `.iloc` &rarr; trabaja con posisiones con enteros (desde 0 a -1) o con arreglos booleanos.

In [ ]:
df_temp=df.drop([1]) #Boto la segunda fila
display(df_temp)

print("Con loc",df_temp.loc[2],"\n",sep="\n") #Aquí 2 es interpretado como "label", será la nueva segunda fila

print("Con iloc",df_temp.iloc[2],"\n",sep="\n") #Aquí como entero, será la tercera fila

Para llamar a varios elementos no secuenciales, usamos también listas

In [ ]:
#Usando listas de elementos
print("Usando listas:")

display(df_temp.loc[[0,3,5]]) #El label 0, 3, 5

display(df_temp.iloc[[0,3,5]]) #Las posiciones asociados (otro label)

#También usando "slices"
print("\nUsando slices ahora:")
print("El dataframe con una fila menos:")
display(df_temp.loc[0:5])
display(df_temp.iloc[0:5])

print("\nEl dataframe original")
display(df.loc[0:5])
display(df.iloc[0:5])
print("En este caso funcionan igual por la naturaleza de este índice. NOTAR se incluye el PRINCIPIO Y EL FINAL!")

Llamando distintas filas y columnas usando `.loc`

In [ ]:
#Mezclando filas y columnas loc
print("Slice y columna - loc",df_temp.loc[2:9,"age"],"\n",sep="\n")

print("Slice y slice - loc")
display(df_temp.loc[2:9,"age":"sex"])

print("\nFilas y columnas - loc")
display(df_temp.loc[[5,3,7],["education","age","country","race"]])

Llamando distintas filas y columnas usando `.iloc`

In [ ]:
#Mezclando filas y columnas iloc (aquí las columnas se preguntan también posicionalmente)

print("Slice y columna - iloc",df_temp.iloc[2:9,0],"\n",sep="\n")

print("Slice y slice - iloc")
display(df_temp.iloc[2:9,0:7])

print("\nFilas y columnas - iloc")
display(df_temp.iloc[[5,3,7],[1,0,5,7]])

Usando slices en `.loc` siempre tiene que ser algo compatible con el índice del df (o al menos *transformable* en este!)

In [ ]:
#Con loc podemos usar slices compatibles con el tipo de índice
display(df_t.loc["20150112":"20160101"])
display(df_t.loc["2015":"2017"])
display(df_t.loc["2015"]) #Todo lo que tenga 2015!

Mezclando posición y label indexing:

In [ ]:
#Con loc, en el índice de las filas podemos indexar posicionalmente el índice:
display(df.loc[df.index[[0,2,5]],["age","education"]])
display(df_t.loc[df_t.index[[0,2,5]],["H","LE"]])
print("\nNotar que df_t.index me entrega los labels que busco en tales posiciones:")
print(df_t.index[[0,2,5]])

#Con iloc, es un poco más enredado, pero podemos llamar el label de la columna:
display(df.iloc[[0,2,5],df.columns.get_indexer(["sex"])])
display(df_t.iloc[[0,2,5],df_t.columns.get_indexer(["H","LE"])])

print("\nNotar que df_t.columns.get_indexer me entrega las posiciones que busco en tales labels:")
print(df_t.columns.get_indexer(["H","LE"]))

Como vemos, indexado básico y `.loc` e `.iloc` gestionan *múltiples casos* de indexado. Si queremos solo conseguir *un* valor, se recomienda usar `.at` y `.iat` (mucho más veloces para esto). Uno ve labels y el otro posiciones respectivamente.

In [ ]:
print(df.at[2,"age"])
print(df.iat[2,0])

print(df_t.at[df_t.index[0],"H"])
print(df_t.iat[0,25])

## Indexar con booleanos

Dentro de las cosas más poderosas y usadas en *Pandas*. Los operadores entre comparaciones son `|` (or), `&` (and), `~` (not) y se deben agrupar con parentesis.

Si estamos trabajando con `series` se trabaja igual que un arreglo numpy:

In [ ]:
#Con series
s=df["age"]
print("Serie original:")
print(s)
print("\nSerie siendo comparada:")
print(s<30)

print("\nEdades menores a 30:")
print(s[s<30]) #Claro ques era lo mismo usar df["age"][df["age"]<30]

print("\nEdades menores a 30 o mayor/igual a 40:")
print(s[(s<30) | (s>= 40)])

print("\nEdades entre 30 y 35:")
print(s[(s>=30) & (s<=35)])

print("\nEdades distintas de 30:")
print(s[s!=30])
print(s[~(s==30)])

Si queremos trabajar con `Dataframes`, basta con entregar un vector booleano del mismo largo que el `index`, por ejemplo, una columna del mismo `Dataframe`!

In [ ]:
print("La columna de edad:",df["age"],sep="\n")
print("\nSi usamos alguna comparación se transforma en 'booleana':",df["age"]>30,sep="\n")
print("\nY con esto podemos FILTRAR el dataframe original:")
display(df[df["age"]>30])

print("Por supuesto que esto puede ser TAN complejo como uno quiera!")

Si queremos resetear el índice basta con usar `.reset_index()`, que almacenará el índice antigüo como una columna, si no queremos que lo guarde, basta con usar *drop=True*

In [ ]:
print("Filtrado:")
display(df[df["age"]>30])

print("Filtrado con nuevo índice y viejo índice como columna:")
display(df[df["age"]>30].reset_index())

print("Filtrado con nuevo índice nada más:")
display(df[df["age"]>30].reset_index(drop=True))

Usando el metodo `.map()` o comprensión de listas podemos realizar criterios más complejos:

In [ ]:
#Método más rápido (busco gente casada)
criterio=df["marital-status"].map(lambda x: x.startswith(" M")) #Ojo que datos parten con un espacio aparentemente
print(criterio)
display(df[criterio])

#Más lento pero idéntico
display(df[[x.startswith(" M") for x in df["marital-status"]]])

#Mezclando esto y otras condiciones
display(df[criterio & (df["education-num"]>=13)]) #Buscando gente casada con algún grado universitario

In [ ]:
#Muy util para seleccionar categorias específicas
criterio=df["relationship"].map(lambda x: x in (" Husband"," Wife"," Own-child"))
df[criterio]

Usando `.loc` e `.iloc` podemos filtrar más aun. Eso si, para `.iloc` se vuelve necesario usar el atributo `.values`, que nos entrega el arreglo numpy con la data almacenada.

In [ ]:
criterio=df["marital-status"].map(lambda x: x.startswith(" M"))
#Con loc es muy directo
display(df.loc[criterio & (df["education-num"]>=13),"age":"education"])

#iloc tira error de la misma manera, tenemos que usar "values"
#display(df.iloc[criterio & (df["education-num"]>=13),[5,3]]) esto tira error!
aux_crit=criterio & (df["education-num"]>=13)
display(df.iloc[aux_crit.values,[5,3]]) #Aquí ningun problema

In [ ]:
print("Notar que:")
print(type(df.values))
print()
display(df.values)
display(df)

## Método where() y masking 

Para `Series`, usar un arreglo booleano usualmente entrega un subset, si queremos que conserve la forma de la data original podemos usar `where`.

In [ ]:
print("La serie original:")
print(df_t["H"])
print()
print("La serie filtrada para valores positivos:")
print(df_t["H"][df_t["H"]>0])

#Ahora usando where
print("\nCon where resulta:")
print(df_t["H"].where(df_t["H"]>0)) #Se reemplazan los valores que no cumplen la condición con NaN!

Ahora hacer este tipo de evaluaciones para un `DataFrame`, conserva el tamaño (está aplicando `where` bajo la mesa). Pero si usamos where, tenemos la posibilidad de escoger otro argumento que reemplazará los valores donde la condición sea falsa

In [ ]:
print("El dataframe original:")
display(df_t)
print("Solo valores positivos, negativos reemplazados con NaN:")
display(df_t[df_t>0]) #Se transforman en NaN
print("Reemplazados con cero:")
display(df_t.where(df_t>0,0))
print("Reemplazados con su valor positivo:")
display(df_t.where(df_t>0,-df_t))

También se puede recibir una función en ambos argumentos

In [ ]:
df_t.where(lambda x:x>0,lambda x:x+100) #Lo primero es lo que busco, lo segundo con que lo reemplazo!

Ahora el método `.mask()` hace lo contrario que `.where()`. O sea reemplaza cuando algo es verdadero.

In [ ]:
df_t.mask(df_t>0) #Basicamente todo al revés!

Más opciones de seleccionar/indexar data en https://pandas.pydata.org/docs/user_guide/indexing.html#

## Otras funciones para ver la data

Hay una serie de funciones que nos permiten visualizar la data. Dentro de las más básicas:
- **df.head(n)** &rarr; nos permite ver las primeras "n" filas (sin n, por default 5), funciona para `Series` y `DataFrame`.
- **df.tail(n)** &rarr; nos permite ver las últimas "n" filas (lo mismo), funciona para `Series` y `DataFrame`.

In [11]:
#Para dataframes
display(df.head())
display(df.head(10))

#Para una serie
print(df["age"].head())

In [12]:
display(df.tail())

display(df["workclass"].tail())

## Estadística

Para tener una descripción estadística básica de nuestra data podemos usar `df.describe()`

In [ ]:
display(df.describe())
df_t.describe()